In [1]:
import pandas as pd
!pip install gensim
from gensim.models import Word2Vec
import nltk
import numpy as np

# Download nltk stopwords
nltk.download('stopwords')
nltk.download('punkt')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.7 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_csv('hate_tweets.csv')

# Tokenize the tweets
df['tokens'] = df['tweet'].apply(lambda x: nltk.word_tokenize(x.lower()))

# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

# Load GloVe embeddings
glove_file = 'glove.6B.50d.txt'
glove_model = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_model[word] = vector

# Define the target word and its replacement
target_word = 'hate'
replace_word = 'love'

# Loop over each tweet
for i, row in df.iterrows():
    tweet = row['tweet']
    tokens = row['tokens']

    # Find the most similar word to the target word
    similar_words = sorted(glove_model.keys(), key=lambda w: np.dot(glove_model[w], glove_model[target_word]))[::-1]
    swap_word = similar_words[0]

    # Replace the target word with the replacement word
    if target_word in tokens:
        tokens[tokens.index(target_word)] = replace_word

    # Replace the most similar word to the target word with the original target word
    if swap_word in tokens:
        tokens[tokens.index(swap_word)] = target_word

    # Recreate the tweet with the modified tokens
    new_tweet = ' '.join(tokens)

    # Print the original and modified tweet
    print(f"Original tweet: {tweet}")
    print(f"Modified tweet: {new_tweet}\n")

Modified tweet: rt @ swaaggyyv : fucking chinks clash clans always fuck . bitch ass kamikaze faggots .

Original tweet: RT @TAXSTONE: Niggaz like all them naked bitches on IG pics NIGGAZ DONT LOVE EM
Modified tweet: rt @ taxstone : niggaz like naked bitches ig pics niggaz dont love em

Original tweet: RT @Tae_Rhodes: &#8220;@kim92493: &#8220;@Tae_Rhodes: @kim92493 @patpatbush uhhh you've been judged&#8221; it happens. #whitepower...I'll hang you nigger&#8221; wo&#8230;
Modified tweet: rt @ tae_rhodes : & # 8220 ; @ kim92493 : & # 8220 ; @ tae_rhodes : @ kim92493 @ patpatbush uhhh 've judged & # 8221 ; happens . # whitepower ... 'll hang nigger & # 8221 ; wo & # 8230 ;

Original tweet: RT @TankTopshotta: A real nigga gone teach his bitch the game not run it on her &#128582;&#128130;&#128520;
Modified tweet: rt @ tanktopshotta : real nigga gone teach bitch game run & # 128582 ; & # 128130 ; & # 128520 ;

Original tweet: RT @Tease: RT or you're a fag. http://t.co/4Ol82rdeqX
Modified tweet

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e803e9f0-f341-4e63-9e5c-ecfa91ec1687' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>